## Setup

In [1]:
# Loading packages and their components
import pandas as pd
import numpy as np
import pickle

from own_functions import train_test_timesplit

# Setting Pandas options
# pd.options.display.max_rows = 999 # For debugging, can be removed later
pd.options.mode.chained_assignment = None  # Disabling the pandas chained assignment warnings

In [2]:
def import_and_preproc():
    # Read in the data
    dengue_features_train = pd.read_csv('data/dengue_features_train.csv')
    dengue_features_test = pd.read_csv('data/dengue_features_test.csv')
    dengue_labels_train = pd.read_csv('data/dengue_labels_train.csv')

    raw_data = [dengue_features_train, dengue_features_test, dengue_labels_train]
    
    # Splitting the data into a San Juan and an Iquitos part
    iq = []
    sj = []
    for item in raw_data:
        sj.append( item[item.city=='sj'] )
        iq.append( item[item.city=='iq'] )

    # Transferring the date column to the label part of the data
    sj[2] = sj[2].join(sj[0]['week_start_date'])
    iq[2] = iq[2].join(iq[0]['week_start_date'])

    # Converting the date column to datetime format
    for i in range(len(sj)):
        sj[i]['week_start_date'] = pd.to_datetime(sj[i]['week_start_date'], format='%Y-%m-%d')  
        iq[i]['week_start_date'] = pd.to_datetime(iq[i]['week_start_date'], format='%Y-%m-%d')
        
    # Putting the date as index
    for i in range(len(sj)):
        sj[i] = sj[i].set_index('week_start_date', drop=False)
        iq[i] = iq[i].set_index('week_start_date', drop=False)
        
    return list([sj[0], sj[1], sj[2], iq[0], iq[1], iq[2]])

data_subsets = import_and_preproc()

## Features in the dataset
City and date indicators
* `city` – City abbreviations: `sj` for San Juan and `iq` for Iquitos
* `week_start_date` – Date given in yyyy-mm-dd format

NOAA's GHCN daily climate data weather station measurements
* `station_max_temp_c` – Maximum temperature
* `station_min_temp_c` – Minimum temperature
* `station_avg_temp_c` – Average temperature
* `station_precip_mm` – Total precipitation
* `station_diur_temp_rng_c` – Diurnal temperature range

PERSIANN satellite precipitation measurements (0.25x0.25 degree scale)
* `precipitation_amt_mm` – Total precipitation

NOAA's NCEP Climate Forecast System Reanalysis measurements (0.5x0.5 degree scale)
* `reanalysis_air_temp_k` – Mean air temperature
* `reanalysis_relative_humidity_percen` – Mean relative humidity
* `reanalysis_specific_humidity_g_per_kg` – Mean specific humidity
* `reanalysis_precip_amt_kg_per_mm` – Total precipitation
* `reanalysis_max_air_temp_k` – Maximum air temperature
* `reanalysis_min_air_temp_k` – Minimum air temperature
* `reanalysis_avg_temp_k` – Average air temperature
* `reanalysis_tdtr_k` – Diurnal temperature range

Satellite vegetation - Normalized difference vegetation index (NDVI) - NOAA's CDR Normalized Difference Vegetation Index (0.5x0.5 degree scale) measurements
* `ndvi_se` – Pixel southeast of city centroid
* `ndvi_sw` – Pixel southwest of city centroid
* `ndvi_ne` – Pixel northeast of city centroid
* `ndvi_nw` – Pixel northwest of city centroid

## Missing value imputation
Since the environmental values for each week are assumed to follow seasonal patterns, they can not be simply replaced with the mean over the entire study. Intstead, missing values in these variables can be replaced with the mean value of the week before and after, or the week before and after that has no missing values.

In [3]:
environmental_vars = [
    'ndvi_ne',
    'ndvi_nw',
    'ndvi_se', 
    'ndvi_sw',
    'precipitation_amt_mm',
    'reanalysis_air_temp_k',
    'reanalysis_avg_temp_k',
    'reanalysis_dew_point_temp_k',
    'reanalysis_max_air_temp_k',
    'reanalysis_min_air_temp_k',
    'reanalysis_precip_amt_kg_per_m2',
    'reanalysis_relative_humidity_percent',
    'reanalysis_sat_precip_amt_mm',
    'reanalysis_specific_humidity_g_per_kg',
    'reanalysis_tdtr_k',
    'station_avg_temp_c',
    'station_diur_temp_rng_c',
    'station_max_temp_c',
    'station_min_temp_c',
    'station_precip_mm'
]

In [4]:
def replace_missing(df, colnames):
    # Store the time index because the code below is index based and needs numbers
    date = df.index
    df = df.reset_index(drop=True)
    for colname in colnames:
        try: # because there are columns that do not occur in all subsets of the dataset
            miss_idx = df[df[colname].isnull()].index.tolist()
            for idx in miss_idx:
                    # Search the nearest week before the week with the missing value
                    # that itself has no missing value
                    before = df.iloc[:idx,:][colname].dropna().tail(1)
                    # The same but for the weeks after the missing value
                    after = df.iloc[idx:,:][colname].dropna().head(1)
                    # Replace the missing value with the mean
                    df[colname][idx] = np.mean([before, after])
        except:
            continue
    # Re-attach the time index and drop the auxiliary index
    df = df.set_index(date, drop=True)
    return df

In [5]:
# Applying the Imputation
for i in range(len(data_subsets)):
    data_subsets[i] = replace_missing(data_subsets[i], environmental_vars)

Check if there are still variables with missing values in our dataset.

In [6]:
# Check if there are still variables with missing values in our dataset.
for subset in data_subsets:
    print(subset.isnull().sum())
    print('---'*10)

city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_pre

## Feature editing
The temperature features from the NCEP Climate Forecast System Reanalysis and those of the weather station are in different units. To have the temperature features in the same units as those of the NCEP Climate Forecast System, the Reanalysis variables will be converted to degrees Celsius. For uniformity, the diurnal temperature range is converted from Celsius to Kelvin, as differences in temperature are expressed in Kelvin. Furthermore, the feature `precipitation_amt_mm` is removed as its values are identical to those of `reanalysis_precip_amt_kg_per_mm`. 

In [7]:
# compare 'reanalysis_sat_precip_amt_mm' and 'precipitation_amt_mm'
for i in range(len(data_subsets)):
    if data_subsets[i].shape[1] > 5:
        print(data_subsets[i][['reanalysis_sat_precip_amt_mm', 'precipitation_amt_mm']].sample(5))

                 reanalysis_sat_precip_amt_mm  precipitation_amt_mm
week_start_date                                                    
1991-06-11                               6.09                  6.09
1994-01-08                              12.60                 12.60
1991-11-12                              78.73                 78.73
2004-02-12                               0.00                  0.00
2006-05-28                               3.70                  3.70
                 reanalysis_sat_precip_amt_mm  precipitation_amt_mm
week_start_date                                                    
2011-12-10                               5.54                  5.54
2008-05-27                               0.76                  0.76
2009-02-19                              11.01                 11.01
2012-12-09                               3.08                  3.08
2012-04-15                              19.20                 19.20
                 reanalysis_sat_precip_amt_mm  p

In [8]:
# apply unit conversion, renaming and dropping 
for i in range(len(data_subsets)):
    if data_subsets[i].shape[1] > 5:
        data_subsets[i] = (
            data_subsets[i]
            .assign(month = lambda df: df.index.month)
            .assign(reanalysis_air_temp_c = lambda df: df['reanalysis_air_temp_k']-273.15)
            .assign(reanalysis_avg_temp_c = lambda df: df['reanalysis_avg_temp_k']-273.15)
            .assign(reanalysis_dew_point_temp_c = lambda df: df['reanalysis_dew_point_temp_k']-273.15)
            .assign(reanalysis_max_air_temp_c = lambda df: df['reanalysis_max_air_temp_k']-273.15)
            .assign(reanalysis_min_air_temp_c = lambda df: df['reanalysis_min_air_temp_k']-273.15)
            .rename(columns={'station_diur_temp_rng_c': 'station_diur_temp_rng_k'})
            .drop(['reanalysis_air_temp_k','reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k',
                   'reanalysis_min_air_temp_k','precipitation_amt_mm'], axis=1)
        )

## Adding the population data

This additional data is available from the Dengue Forecasting [website](https://dengueforecasting.noaa.gov/), from which the data provided by DrivenData originates.

In [9]:
# import the population data for sj and iq
def load_pop(filename):
    ser = (
        pd.read_csv(filename)
        .assign(year = lambda df: df.Year) # to have a same-name column with the other dataframes
        .assign(Year = lambda df: pd.to_datetime(df.Year, format='%Y'))
        .set_index('Year', drop=True)
    )
    return ser
sj_pop = load_pop('data/San_Juan_Population_Data.csv')
iq_pop = load_pop('data/Iquitos_Population_Data.csv')

In [10]:
def merge_pop(df, pop):
    merged = pd.merge(df, pop, how='left', on='year')
    merged = merged.rename(columns={'Estimated_population': 'population'})
    merged = merged.set_index('week_start_date', drop=True)
    merged.population = merged.population.interpolate().round().astype(int)
    return merged

In [11]:
data_subsets[2].head()

,city,year,weekofyear,total_cases,week_start_date
week_start_date,,,,,
1990-04-30,sj,1990,18,4,1990-04-30
1990-05-07,sj,1990,19,5,1990-05-07
1990-05-14,sj,1990,20,4,1990-05-14
1990-05-21,sj,1990,21,3,1990-05-21
1990-05-28,sj,1990,22,6,1990-05-28


In [12]:
data_subsets[0] = merge_pop(data_subsets[0], sj_pop)
data_subsets[1] = merge_pop(data_subsets[1], sj_pop)
data_subsets[2] = data_subsets[2].set_index('week_start_date', drop=True)
data_subsets[3] = merge_pop(data_subsets[3], iq_pop)
data_subsets[4] = merge_pop(data_subsets[4], iq_pop)
data_subsets[5] = data_subsets[5].set_index('week_start_date', drop=True)

In [13]:
# add week_start_date as seperate column to both test datasets and delete index name (sj_test and iq_test)
# happen for the other dataframes in the train-test-split
data_subsets[1]['week_start_date'] = data_subsets[1].index
data_subsets[4]['week_start_date'] = data_subsets[4].index
data_subsets[1].index.name = None
data_subsets[4].index.name = None

In [14]:
# Splitting the data into their parts
sj_features_train, \
sj_test, \
sj_labels_train, \
iq_features_train, \
iq_test, \
iq_labels_train = data_subsets

In [15]:
pickle.dump(data_subsets, open('cleaned_data.pickle', 'wb'))

## Train test split
To evaluate forecasting models, a train and a seperate test dataset (with actual values for the number of cases) is needed. Therefore the given "train" datasets for both cities is split into a `train_train` (75% of the data) and a `train_test` (25% of the data) set. 

#### Exclude data before 2002 from Iquitos
The total number of cases form Iquitos only contain single values. After 01.01.2002 the total number of cases increases clearly, probably due to a difference in the reposting system or counting system. Consequently the values before 2002 will be excluded from the dataset used for modeling.

In [16]:
# remove entries in IQ data until 2002 (data excluded from modeling)
data_subsets[3] = data_subsets[3]['2002':]
data_subsets[5] = data_subsets[5]['2002':]

In [17]:
# split given data (features and label) into test and train datasets
def split_dataset(data_subsets):
    data_subsets_splitted = []
    for i in [0, 2, 3, 5]:
        train, test = train_test_timesplit(data_subsets[i])
        data_subsets_splitted.append(train)
        data_subsets_splitted.append(test)
    return data_subsets_splitted

data_subsets_splitted = split_dataset(data_subsets)

In [18]:
# splitting the data into their parts
sj_features_train_train, \
sj_features_train_test, \
sj_labels_train_train, \
sj_labels_train_test, \
iq_features_train_train, \
iq_features_train_test, \
iq_labels_train_train, \
iq_labels_train_test = data_subsets_splitted

Combine data for one `train_train` and `train_test` set for each city

In [19]:
# join feature and label dataset
sj_train_train = sj_features_train_train.join(sj_labels_train_train['total_cases'])
sj_train_test = sj_features_train_test.join(sj_labels_train_test['total_cases'])
iq_train_train = iq_features_train_train.join(iq_labels_train_train['total_cases'])
iq_train_test = iq_features_train_test.join(iq_labels_train_test['total_cases'])

# combine all six datasets (train_train, train_test and test)
data_subsets_splitted_joined = [sj_train_train, sj_train_test, sj_test, iq_train_train, iq_train_test, iq_test]

In [20]:
# save the splitted data subsets in a pickle
pickle.dump(data_subsets_splitted_joined, open('splitted_joined_data.pickle', 'wb'))